In [4]:
import numpy as np
import math
import scipy.constants
import matplotlib.pyplot as plt

In [167]:
class Ruku4:
    
    def __init__(self, function, varss, x_in, x_fin):
        self.calc_func = function
        self.vars = varss
        self.x_in = x_in
        self.x_fin = x_fin
        self.time_to_finish = False
   
    ## Evaluate the function at different points ##
    def eval_func(self, calc_func, x_val, y_vals):
        calc_func1 = np.array([])
        x1 = x_val
        for i in range(len(y_vals)): exec(f'x{i+2}=y_vals[i]')
        
        for i in calc_func:
            try: evaled = eval(i)
            except:
                self.time_to_finish = True
                calc_func1 = 0
                return calc_func1
            calc_func1 = np.append(calc_func1, evaled)
            
        return calc_func1

    ## Checks If some inputs are zero or infinity and corrects accordingly ##
    def check_values(self, x_in, y_ins):
        with np.errstate(divide ='ignore', invalid ='ignore'):
            if x_in == 0: x_in = 1e-15
            for num in range(len(y_ins)):
                if y_ins[num] == 0: y_ins[num] == 1e-16
                elif y_ins[num] == np.inf: y_ins[num] == 1e16
        return x_in,y_ins
    
    ## Return the slopes at different points ##
    def find_slopes(self, calc_func, h, x_in, y_ins):
        x_in, y_ins=self.check_values(x_in, y_ins)
        k1 = h*self.eval_func(calc_func, x_val = x_in,y_vals = y_ins)
        k2 = h*self.eval_func(calc_func, x_val = x_in + h/2,y_vals = y_ins + k1/2)
        k3 = h*self.eval_func(calc_func, x_val = x_in + h/2,y_vals = y_ins + k2/2)
        k4 = h*self.eval_func(calc_func, x_val = x_in + h,y_vals = y_ins + k3)
        return k1,k2,k3,k4
    
    def ruku4_calc(self, calc_func, h, x, y):
        k1, k2, k3, k4 = self.find_slopes(calc_func, h, x, y)
        y = y + (k1 + 2*k2 + 2*k3 + k4)/6
        return y 
    
    ## Runge Kutta 4th Order Method ##             
    def Ruku4(self, h, y_ins, x_req = 0, full_output = False, plot = False):
        num = 0
        x_pt = {}
        x, y = self.x_in, y_ins
        for i in range(1, (1 + len(y)) + 1): x_pt[i] = np.array([])
        
        while x <= x_fin and y[0] >= 0 and y[1] >= 0 and not self.time_to_finish:
           
            if x == x_req or x == self.x_fin or x == self.x_in or full_output:
                print('\n {} = '.format(self.vars[0]), x, end =' ')
                for j in range(len(y)): print(', {} = '.format(self.vars[j+1]), y[j], end =' ')
                print(', h = ',h)
            
            y_half = self.ruku4_calc(self.calc_func, h/2, x, y)
            y_half = self.ruku4_calc(self.calc_func, h/2, x, y_half)
            y = self.ruku4_calc(self.calc_func, h, x, y)

            if np.sum(abs((y_half-y)/y)) > 1e-5:
                h /= 2
            if np.sum(abs((y_half-y)/y)) <= 1e-5:
                h *= 2
            if x + h > self.x_fin and x < self.x_fin:
                h = self.x_fin - x 
            
            x_pt[1] = np.append(x_pt[1], x) 
            for i in range(2, (1 + len(y)) + 1): x_pt[i] = np.append(x_pt[i], y[i-2]) 
            num += 1
            x += h
            
        self.time_to_finish = False
        
        if plot == True:
            fig, ax = plt.subplots(len(y))
            fig.subplots_adjust(top = 4, bottom = 2, left = 0, right = 1, hspace = 0.25)
            for i in range(2, (1 + len(y)) + 1):
                y, x = self.vars[i-1], self.vars[0]
                ax[i-2].plot(x_pt[1], x_pt[i], color = (1, 0.5, 0), alpha = 1)
                ax[i-2].set_title(f"{y} vs {x}")
                ax[i-2].set_xlabel(f"{x}")
                ax[i-2].set_ylabel(f"{y}")
                
        return x_pt[2][-1], x_pt[1][-1]

In [168]:
K = 5.4*10**3
V = 5/3
G = scipy.constants.G
c = scipy.constants.c
pi = math.pi

h = 0.1
x_in = 0.0001
x_fin = 12988
rho_c = 1e18
            #dm/dr               #drho/dr
sample = [f'x3*4*({pi})*x1**2', f'-({G})*(x3+({K})*x3**({V})/({c})**2)/(({K})*({V})*x3**(({V})-1))*(x2+4*({pi})*x1**3*({K})*x3**({V})/({c})**2)/(x1**2*(1-2*({G})*x2/(({c})**2*x1)))']
initial = np.array([4/3*pi*x_in**3*rho_c,rho_c])

In [169]:
RuKu4 = Ruku4(function = sample, varss = ['radius', 'mass', 'density'], x_in = x_in, x_fin = x_fin)
RuKu4.Ruku4(h = h, y_ins = initial, plot = True)

In [140]:
x_in, x_fin = 0.0001, 50000
rho_c = 1e17
rho_range = 100
m_vs_rho = np.array([])
rho = np.array([])
r_vs_rho = np.array([])

RuKu4 = Ruku4(function = sample, varss = ['radius', 'mass', 'density'], x_in = x_in, x_fin = x_fin)
for i in range(rho_range):
    ## don't use numpy array for 'initial'
    print(f'\n at rho = {(i*10+1)*rho_c}')
    initial = [4/3*pi*x_in**3*((i*10+1)*rho_c),(i*10+1)*rho_c]
    m_at_rho,r_at_rho = RuKu4.Ruku4(h = h, y_ins = initial)
    m_vs_rho = np.append(m_vs_rho,m_at_rho/(1.989*10**30))
    r_vs_rho = np.append(r_vs_rho,r_at_rho)

In [166]:
fig, ax = plt.subplots(2)
fig.subplots_adjust(top = 4, bottom = 2, left = 1, right = 2, hspace = 0.25, wspace = 0)

ax[0].plot(np.arange(1, rho_range*10, 10), m_vs_rho, color=(0, 0, 0), alpha = 1)
ax[0].set_xlabel(r'$\rho$({})'.format(rho_c))
ax[0].set_ylabel(r'Mass($M_{sun}$)')
ax[0].set_title(r'Mass vs $\rho$')

ax[1].plot(m_vs_rho, r_vs_rho, color = (1, 0, 0), alpha = 1)
ax[1].set_xlabel(r'Mass($M_{sun}$)')
ax[1].set_ylabel('Radius(m)')
ax[1].set_title('Radius vs Mass')